This file reads data in csv format and uploads to local POSTGRESS database.

In [1]:
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd
import numpy as np
import os
from collections import defaultdict

os.chdir('/Users/deniz/Research/Insight_Churn/')

In [2]:
#In Python: Define a database name
dbname = 'CHURN'
username = 'deniz'

In [3]:
# 'engine' is connection to postgres database
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print engine.url

postgres://deniz@localhost/CHURN


In [4]:
# create a database (if it doesn't exist)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

True


Below we read each csv file one by one (making sure date/boolean fields are parsed properly) and then save to database.

In [ ]:
path = '/Users/deniz/Research/Insight_Churn/data/'

In [ ]:
# HOLIDAYS (TIME OFF)
dtypes = {'date': 'str', 'shift':'str'}
parse_dates = ['date']
holidays = pd.read_csv(path + 'provider_holidays.csv',sep=';',
                      dtype=dtypes, parse_dates=parse_dates)
holidays.loc[holidays['hour']==8,'hour']='morning' 
holidays.loc[holidays['hour']==13,'hour']='afternoon'
holidays.rename(columns = {'hour':'shift'}, inplace = True)
# Remove future time-off data
holidays = holidays[holidays['date']<'2017-01-20']
holidays.to_sql('holidays', engine, if_exists='replace')

print holidays.columns.to_series().groupby(holidays.dtypes).groups
holidays.head()

In [ ]:
# PROVIDERS
# read a database from CSV and load it into a pandas dataframe
dtypes = {'started_datetime': 'str', 'finished_datetime': 'str','birthday':'str'}
parse_dates = ['started_datetime', 'finished_datetime','birthday']
providers = pd.read_csv(path + 'providers.csv',sep=';',
                        dtype=dtypes, parse_dates=parse_dates)
providers.to_sql('providers', engine, if_exists='replace')
print providers.columns.to_series().groupby(providers.dtypes).groups
providers.head()

In [ ]:
# SERVICES
dtypes = {'date': 'str', 'planned_entry_datetime_cache': 'str','provider_checked_in':'str',
          'planned_exit_datetime_cache':'str','provider_checked_out':'str'}
parse_dates = ['date', 'planned_entry_datetime_cache','provider_checked_in','planned_exit_datetime_cache',
               'provider_checked_out']

services = pd.read_csv(path + 'services.csv', sep=';',
                                               dtype=dtypes, parse_dates=parse_dates)
services.loc[services['morning']=='t','morning']=True
services.loc[services['morning']=='f','morning']=False
services.loc[services['afternoon']=='t','afternoon']=True
services.loc[services['afternoon']=='f','afternoon']=False
services.loc[services['first_time_of_provider_in_address_cache']=='t','first_time_of_provider_in_address_cache']=True
services.loc[services['first_time_of_provider_in_address_cache']=='f','first_time_of_provider_in_address_cache']=False
services.to_sql('services', engine, if_exists='replace')
print services.columns.to_series().groupby(services.dtypes).groups
print services['status'].unique()
services.head()

In [ ]:
# PAYROLLS
payrolls = pd.read_csv(path + 'payrolls.csv', sep=';')
payrolls.to_sql('payrolls', engine, if_exists='replace')
print payrolls.columns.to_series().groupby(payrolls.dtypes).groups
payrolls.head()

In [ ]:
# PAYROLL TERMS
dtypes = {'start_date': 'str', 'end_date': 'str'}
parse_dates = ['start_date', 'end_date']
payroll_terms = pd.read_csv(path + 'payroll_terms.csv', sep=';',
                            dtype=dtypes, parse_dates=parse_dates)
payroll_terms.to_sql('payroll_terms', engine, if_exists='replace')
print payroll_terms.columns.to_series().groupby(payroll_terms.dtypes).groups
payroll_terms.head()

In [ ]:
# BANK TRANSACTIONS
dtypes = {'date': 'str'}
parse_dates = ['date']
bank_transactions = pd.read_csv(path + 'bank_transactions.csv', sep=';', 
                                dtype=dtypes, parse_dates=parse_dates, skiprows = [22131, 24684, 4174, 11432])
bank_transactions.to_sql('bank_transactions', engine, if_exists='replace')
print bank_transactions.columns.to_series().groupby(bank_transactions.dtypes).groups
bank_transactions.head()

In [ ]:
# BANK TRANSACTION TYPES
bank_transaction_types = pd.read_csv(path + 'bank_transaction_types.csv', sep=';')
bank_transaction_types.to_sql('bank_transaction_types', engine, if_exists='replace')
print bank_transaction_types.columns.to_series().groupby(bank_transaction_types.dtypes).groups
bank_transaction_types.head()

In [ ]:
# SERVICE RATINGS
dtypes = {'created_at': 'str'}
parse_dates = ['created_at']
service_reviews = pd.read_csv(path + 'service_reviews.csv', sep=';',
                              dtype=dtypes, parse_dates=parse_dates)
service_reviews.drop_duplicates(subset=['service_id'], keep='last', inplace=True)
service_reviews.to_sql('service_reviews', engine, if_exists='replace')
print service_reviews.columns.to_series().groupby(service_reviews.dtypes).groups
service_reviews.head()

In [ ]:
# PROVIDER RATINGS
dtypes = {'created_at': 'str'}
parse_dates = ['created_at']
provider_reviews = pd.read_csv(path + 'provider_satisfaction_checks.csv',
                               sep=';', dtype=dtypes, parse_dates=parse_dates)
provider_reviews.to_sql('provider_reviews', engine, if_exists='replace')
print provider_reviews.columns.to_series().groupby(provider_reviews.dtypes).groups
provider_reviews.head()

In [ ]:
# BANS
dtypes = {'created_at': 'str'}
parse_dates = ['created_at']
bans = pd.read_csv(path + 'bans.csv',
                               sep=';', dtype=dtypes, parse_dates=parse_dates)
bans.to_sql('bans', engine, if_exists='replace')
print bans.columns.to_series().groupby(bans.dtypes).groups
bans.head()

In [ ]:
# MISBEHAVIOURS
dtypes = {'date': 'str'}
parse_dates = ['date']
misbehaviors = pd.read_csv(path + 'misbehaviors.csv',
                               sep=';', dtype=dtypes, parse_dates=parse_dates, skiprows = [659, 11497])
misbehaviors.to_sql('misbehaviors', engine, if_exists='replace')
print misbehaviors.columns.to_series().groupby(misbehaviors.dtypes).groups
misbehaviors.head()

In [ ]:
# MISBEHAVIOUR TYPES
misbehavior_types = pd.read_csv(path + 'misbehavior_types.csv', sep=';')
misbehavior_types.to_sql('misbehavior_types', engine, if_exists='replace')

misbehavior_types.loc[misbehavior_types['severe']=='t','severe']=True
misbehavior_types.loc[misbehavior_types['severe']=='f','severe']=False
print misbehavior_types.columns.to_series().groupby(misbehavior_types.dtypes).groups
misbehavior_types.head()

In [ ]:
# CLIENTS
clients = pd.read_csv(path + 'clients.csv', sep=';')
clients.to_sql('clients', engine, if_exists='replace')
# work type
clients = clients.rename(columns={'nit': 'office'})
clients.loc[clients['office']=='t','office']=True
clients.loc[clients['office']=='f','office']=False
print clients.columns.to_series().groupby(clients.dtypes).groups
clients.head()

In [ ]:
# PROVIDER STATS 
dtypes = {'date': 'str','created_at':'str', 'updated_at':'str'}
parse_dates = ['date', 'created_at', 'updated_at']
provider_stats = pd.read_csv(path + 'provider_stats.csv', sep=';', 
                             dtype=dtypes, parse_dates=parse_dates)
provider_stats.to_sql('provider_stats', engine, if_exists='replace')

print provider_stats.columns.to_series().groupby(provider_stats.dtypes).groups

In [ ]:
# TERMINATION
terminations = pd.read_csv(path + 'providers_updated.csv',sep=';',
                        usecols=['id','finish_type','reason_finished'])
terminations.fillna('missing',inplace=True)
terminations.to_sql('terminations', engine, if_exists='replace')
print terminations.columns.to_series().groupby(terminations.dtypes).groups
terminations.head()

Let's print out all data available in the database:

In [6]:
for i in engine.table_names():
    print i 

clients
provider_stats
terminations
holidays
providers
services
payrolls
payroll_terms
bank_transactions
bank_transaction_types
service_reviews
provider_reviews
bans
misbehaviors
misbehavior_types
